In [11]:
import numpy as np 
import pandas as pd 
import pickle
import snap

In [12]:
pkl_file = open('EPL_Games.pkl', 'rb')
Data = pickle.load(pkl_file)
pkl_file.close()

In [5]:
Data[442372]

[(4, 3),
 {(1881.0, 7933.0): 5,
  (1881.0, 17974.0): 5,
  (1881.0, 18753.0): 0,
  (1881.0, 21060.0): 0,
  (1881.0, 27334.0): 4,
  (1881.0, 27341.0): 0,
  (1881.0, 28301.0): 2,
  (1881.0, 37939.0): 0,
  (1881.0, 40799.0): 0,
  (1881.0, 42727.0): 1,
  (1881.0, 42758.0): 5,
  (1881.0, 44488.0): 2,
  (1881.0, 132931.0): 0,
  (1881.0, 'Gain'): 9,
  (1881.0, 'Goal'): 0,
  (1881.0, 'Loss'): 12,
  (1881.0, 'Shot'): 0,
  (7933.0, 1881.0): 4,
  (7933.0, 17974.0): 5,
  (7933.0, 18753.0): 2,
  (7933.0, 21060.0): 7,
  (7933.0, 27334.0): 2,
  (7933.0, 27341.0): 2,
  (7933.0, 28301.0): 5,
  (7933.0, 37939.0): 1,
  (7933.0, 40799.0): 4,
  (7933.0, 42727.0): 3,
  (7933.0, 42758.0): 2,
  (7933.0, 44488.0): 6,
  (7933.0, 132931.0): 0,
  (7933.0, 'Gain'): 6,
  (7933.0, 'Goal'): 0,
  (7933.0, 'Loss'): 3,
  (7933.0, 'Shot'): 0,
  (17974.0, 1881.0): 7,
  (17974.0, 7933.0): 5,
  (17974.0, 18753.0): 1,
  (17974.0, 21060.0): 0,
  (17974.0, 27334.0): 6,
  (17974.0, 27341.0): 1,
  (17974.0, 28301.0): 5,
  (17974.

In [29]:
def Create_Graph(Data, match_id):
    
    G_home = snap.TNEANet.New()
    G_away = snap.TNEANet.New()
    game_data = Data[match_id]
    
    players_home = game_data[7]
    players_away = game_data[8]
    teams=game_data[0]
    
    home_weights = game_data[5]
    away_weights = game_data[6]
    
    for i in players_home: G_home.AddNode(int(i))
        
    for i in players_away: G_away.AddNode(int(i))
    
    # Have 0, 1, 2 correspond to shot, gain, and loss respectively assuming player id is > 2
    for i in [0, 1, 2]:
        G_home.AddNode(i)
        G_away.AddNode(i)
    
    # Add attribute for weight
    G_home.AddFltAttrE("weight")
    G_away.AddFltAttrE("weight")

    eid = 1
    for key in home_weights.keys():
        
        src, dst = key
        
        if dst == 'Shot': dst = 0
        if dst == 'Gain': dst = 1
        if dst == 'Loss': dst = 2
        
        if dst == 'Goal': continue
            

        G_home.AddEdge(int(src), int(dst), eid)
        
        G_home.AddFltAttrDatE(eid, home_weights[key], "weight")
        
        eid += 1
    
    eid = 1
    for key in away_weights.keys():
        
        src, dst = key
        
        if dst == 'Shot': dst = 0
        if dst == 'Gain': dst = 1
        if dst == 'Loss': dst = 2
            
        if dst == 'Goal': continue


        G_away.AddEdge(int(src), int(dst), eid)
        
        G_away.AddFltAttrDatE(eid, away_weights[key], "weight")
        
        eid += 1
    
    return G_home, G_away
        
    

In [73]:
pkl_file = open('simple_features_by_game.pkl', 'rb')
sf_game = pickle.load(pkl_file)
pkl_file.close()

In [112]:
pkl_file = open('simple_features_by_team_game.pkl', 'rb')
sf_team = pickle.load(pkl_file)
pkl_file.close()

In [120]:
Node_Features = {}
Graph_Features_game = {}
Graph_Features_team = {}

ind = 1
n = len(Data.keys())
for k in Data.keys():
    
    print(str(ind)+"/"+str(n))
    
    G1, G2 = Create_Graph(Data, k)
    
    teams = Data[k][0]
    
    F1 = snap.TIntFltVH() 
    F2 = snap.TIntFltVH() 

    snap.node2vec(G1, 1, 1, 128, 80, 10, 10, 1, False, F1)
    snap.node2vec(G2, 1, 1, 128, 80, 10, 10, 1, False, F2)
    
    Node_Features[(k,'Home')] = F1
    Node_Features[(k,'Away')] = F2
    
    graph_vec = 0
    for i in F1:
        graph_vec += np.array([j for j in F1[i]])
    graph_vec_f1 = graph_vec/float(len(F1))
    
    graph_vec = 0
    for i in F2:
        graph_vec += np.array([j for j in F2[i]])
    graph_vec_f2 = graph_vec/float(len(F2))
    
    r = sf_game[sf_game.match_id==k].result.values[0]
    gd = sf_game[sf_game.match_id==k].goal_dif.values[0]
    
    r_th = sf_team[(sf_team.match_id==k) & (sf_team.home ==1)].result.values[0]
    g_th = sf_team[(sf_team.match_id==k) & (sf_team.home ==1)].goals.values[0]
    
    r_ta = sf_team[(sf_team.match_id==k) & (sf_team.home ==0)].result.values[0]
    g_ta = sf_team[(sf_team.match_id==k) & (sf_team.home ==0)].goals.values[0]
    
    Graph_Features_game[k] = np.hstack([k, graph_vec_f1, graph_vec_f2, teams[0], teams[1], gd, r])
    Graph_Features_team[(k, 'Home')] = np.hstack([k, graph_vec_f1, teams[0], 1, g_th, r_th])
    Graph_Features_team[(k, 'Away')] = np.hstack([k, graph_vec_f2, teams[1], 0, g_ta, r_ta])
    
    ind+=1


1/380
2/380
3/380
4/380
5/380
6/380
7/380
8/380
9/380
10/380
11/380
12/380
13/380
14/380
15/380
16/380
17/380
18/380
19/380
20/380
21/380
22/380
23/380
24/380
25/380
26/380
27/380
28/380
29/380
30/380
31/380
32/380
33/380
34/380
35/380
36/380
37/380
38/380
39/380
40/380
41/380
42/380
43/380
44/380
45/380
46/380
47/380
48/380
49/380
50/380
51/380
52/380
53/380
54/380
55/380
56/380
57/380
58/380
59/380
60/380
61/380
62/380
63/380
64/380
65/380
66/380
67/380
68/380
69/380
70/380
71/380
72/380
73/380
74/380
75/380
76/380
77/380
78/380
79/380
80/380
81/380
82/380
83/380
84/380
85/380
86/380
87/380
88/380
89/380
90/380
91/380
92/380
93/380
94/380
95/380
96/380
97/380
98/380
99/380
100/380
101/380
102/380
103/380
104/380
105/380
106/380
107/380
108/380
109/380
110/380
111/380
112/380
113/380
114/380
115/380
116/380
117/380
118/380
119/380
120/380
121/380
122/380
123/380
124/380
125/380
126/380
127/380
128/380
129/380
130/380
131/380
132/380
133/380
134/380
135/380
136/380
137/380
138/380
139/

In [133]:

GF_g = []
GF_t = []
for k in Graph_Features_game:
    L = Graph_Features_game[k]
    if len(GF_g) == 0:
        GF_g = L
    else:
        GF_g = np.vstack([GF_g, L])
    
for k in Graph_Features_team:
    L = Graph_Features_team[k]
    if len(GF_t) == 0:
        GF_t = L
    else:
        GF_t = np.vstack([GF_t, L])
    
    
df_GF_g = pd.DataFrame(data=GF_g)
df_GF_t = pd.DataFrame(data=GF_t)

In [136]:
df_GF_g.rename(columns={260: 'result', 259: 'goal_dif', 258: 'away_team', 257: 'home_team', 0: 'match_id'}, inplace=True)

In [138]:
df_GF_t.rename(columns={132: 'result', 131: 'goals', 130: 'home', 129: 'team_id', 0: 'match_id'}, inplace=True)

In [140]:
output = open('node2vec_by_team_game.pkl', 'wb')
pickle.dump(df_GF_t, output)
output.close()

output = open('node2vec_by_game.pkl', 'wb')
pickle.dump(df_GF_g, output)
output.close()

In [116]:
sf_team[sf_team.match_id==442373]

,match_id,team_id,shot_rate,gain_rate,loss_rate,pass_rate,max_pass,min_pass,number_players,home,goals,result
748,442373.0,20.0,0.016435,0.075086,0.156246,0.043317,0.129827,0.027841,14.0,1.0,1.0,0.0
749,442373.0,110.0,0.012629,0.073028,0.144143,0.019677,0.088851,0.019524,14.0,0.0,1.0,0.0


In [121]:
np.horzcat(np.array([]), np.array([2,3,4]))

AttributeError: 'module' object has no attribute 'horzcat'